In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import numpy as np
import glob
import json
import cv2
import copy

In [2]:
# import os
# os.chdir('/content/drive/MyDrive/Datasets')

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import json
import cv2
import copy

from tqdm.autonotebook import tqdm
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset

<ipython-input-3-e1e031804310>:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
def get_video_files(path):
  video_files =  glob.glob(path+'*.MP4')

  # print(video_files)

  frame_count = []
  for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    if(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))<150):
      video_files.remove(video_file)
      continue
    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))
  print("frames" , frame_count)
  print("Total number of videos: " , len(frame_count))
  print('Average frame per video:',np.mean(frame_count))

  return video_files

In [5]:
# !pip install dlib==19.22.1

In [6]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 10.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=178b5b53a5a9da012881a243431750ae3d992437e972bd33644d929423e3d355
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [7]:
import dlib

print("Is dlib compiled with CUDA:", dlib.DLIB_USE_CUDA)
print("CUDA Device Count:", dlib.cuda.get_num_devices())

Is dlib compiled with CUDA: True
CUDA Device Count: 1


In [8]:
import face_recognition
# face_recognition.face_distance([], [], use_gpu=False)

In [9]:
# to extract frame
def frame_extract(path):
  vidObj = cv2.VideoCapture(path)
  success = 1
  while success:
      success, image = vidObj.read()
      if success:
          yield image

In [10]:
# process the frames
def create_face_videos(path_list,out_dir):
  already_present_count =  glob.glob(out_dir+'*.MP4')
  print("No of videos already present " , len(already_present_count))
  for path in tqdm(path_list):
    out_path = os.path.join(out_dir,path.split('/')[-1])
    file_exists = glob.glob(out_path)
    if(len(file_exists) != 0):
      print("File Already exists: " , out_path)
      continue
    frames = []
    flag = 0
    face_all = []
    frames1 = []
    out = cv2.VideoWriter(out_path,cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112,112))
    for idx,frame in enumerate(frame_extract(path)):
      #if(idx % 3 == 0):
      if(idx <= 150):
        frames.append(frame)
        if(len(frames) == 4):
          faces = face_recognition.batch_face_locations(frames)
          for i,face in enumerate(faces):
            if(len(face) != 0):
              top,right,bottom,left = face[0]
            try:
              out.write(cv2.resize(frames[i][top:bottom,left:right,:],(112,112)))
            except:
              pass
          frames = []
    try:
      del top,right,bottom,left
    except:
      pass
    out.release()

In [11]:
# youtube real data
video_files = get_video_files('/content/drive/MyDrive/videos_to_test/')
create_face_videos(video_files,'/content/drive/MyDrive/preprocessed_test/')


frames [374]
Total number of videos:  1
Average frame per video: 374.0
No of videos already present  0


  0%|          | 0/1 [00:00<?, ?it/s]